# EAC 4

**Partint dels notebooks de la lliçó 3, desenvolupeu un codi que:**

1. Instal·li i configuri totes les dependències necessàries de LangChain i Hugging Face a Google Colab, resolent els conflictes habituals de paquets. Consell: fixar requests==2.32.4 per compatibilitat amb Colab.
2. Utilitzi un fitxer amb text sobre RAG o un altre tema tècnic (p.ex. IA, machine learning, etc.).
3. Indexi el text amb RecursiveCharacterTextSplitter, embeddings de sentence-transformers/all-MiniLM-L6-v2 i un índex FAISS.
4. Carregui un model lliure de text a text, com ara google/flan-t5-base o flan-t5-small, mitjançant HuggingFacePipeline.
5. Construeixi una cadena RAG combinant el retriever i el prompt de generació, fent que el model respongui preguntes en català basant-se en el context recuperat.
6. Faci tres proves de pregunta-resposta sobre el contingut del corpus i mostri les respostes generades.
(Opcional) Calculi una petita mètrica automàtica (proxy) entre 0 i 1 basada en paraules clau per autoavaluar la qualitat de les respostes.

## Instalación de las librerías

In [ ]:
!pip install -U requests==2.32.4 -q # Compatibilidad con Google Colab
!pip install -U langchain==0.3.7 -q # Marco de trabajo LangChain
!pip install -U langchain-community==0.3.7 -q
!pip install -q sentence-transformers -q #Framework para representaciones vectoriales de frases
!pip install PyPDF2 -q # Lector de PDFs
!pip install faiss-cpu -q # Facebook AI Similarity Search


## Lectura del PDF
Para extraer todo el texto del documento se ha de ir página por página e ir dejando el contenido en la variable text.

In [ ]:
from PyPDF2 import PdfReader

# Cargamos el fichero en PDF
pdf_file_obj = open('inteligencia_artificial.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

In [ ]:
text = ""
# Iteramos página a página
for page in pdf_reader.pages:
    text += page.extract_text()

In [ ]:
# Mostramos una parte del texto
text[16000:18000]

'unció cognitiva–, es coneix com a regla de Hebb pels treballs del\nneuropsicòleg canadenc Donald O. Hebb publicats en el seu llibre de 1949\nThe organization of behavior . Algunes tècniques d’intel ·ligència artiﬁcial com\nels mètodes d’ aprenentatge supervisat també es basen en regles similars que per-\nmeten modiﬁcar de manera adaptativa la manera en què el sistema artiﬁcial\nprocessa la informació.\nLa intel·ligència artiﬁcial (IA) és una disciplina acadèmica relacionada amb\nla teoria de la computació que té l’objectiu d’emular algunes de les facultats\nintel·lectuals humanes en sistemes artiﬁcials. Amb intel·ligència humana ens\nreferim típicament a processos de percepció sensorial (visió, audició, etc.) i\nals processos consegüents de reconeixement de patrons, per la qual cosa lesCC-BY-NC-ND •PID_00174125 11 Intel.ligència artiﬁcial avançada\naplicacions més habituals de la IA són el tractament de dades i la identiﬁca-\nció de sistemes. Això no exclou que la IA, des dels seus in

## Creamos las porciones de texto (Chunks)
Se divide el documento en frases o parrafos. La medida de cada porción será de 300 carácteres, con un solapamiento de 50 carácteres, ya que flan-t5-large tiene una ventana de contexto de 512 tokens(2000 carácteres), y con este tamaño no se supera.

In [ ]:
# Cargamos la librería responsable de la división del texto
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Dividimos el texto en fragmentos más pequeños(chunks)
# Hacemos porciones de 300 carácteres y solapamientos de 50 carácteres

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Carácteres
    chunk_overlap=50, # Solapamientos
    length_function=len
    )

In [ ]:
# Pasamos el texto al objeto
chunks = text_splitter.split_text(text)

In [ ]:
# Mostramos la longitud de las porciones en la cual se ha dividido el texto
len(chunks)

1859

In [ ]:
# Mostramos porción número 130 contiene el siguiente texto
chunks[130]

'El sistema d’IA modularia l’execució de les diferents accions, incrementant la\nprobabilitat d’aquelles que presentin un millor rendiment i restringint les que\nno comportin una millora de la funció objectiu. Un àrea afí a la robòtica és'

## Creamos las incrustaciones
Una vez tenemos el texto dividido se crea una incrustación(embedding) para cada porción. Después utilizamos las incrustaciones hechas con la libreria (Facebook AI Sentence Search) para que a partir de una pregunta nos retorne las porciones de texto más relevantes del documento, las cuales serán utilizadas para pasarlas al LLM.

In [ ]:
# Importamos las librerías para hacer las inscrustaciones
from langchain.embeddings import HuggingFaceEmbeddings

# Escogemos la versión 'MiniLM-L6-v2
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-1236155258.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
"""
FAISS es una biblioteca desarrollada por meta para buscar rapidamente los vectores más similares dentro de una base de datos.
"""

# Cargamos la librería FAISS
from langchain.vectorstores import FAISS

"""
Creamos una base de datos vectorial a partir de las porciones del texto.
"""
# Pasamos a FAISS las porciones del texto más los embeddings
knowledge_base = FAISS.from_texts(chunks, embeddings)

## Preguntamos al documento
Utilizamos el LLM para obtener la mejor respuesta hecha a partir de las porciones de texto que se han obtenido.

In [ ]:
# Cargamos el modelo 'flan-t5-large' de Hugging Face

from langchain.llms import HuggingFacePipeline
from transformers import pipeline

nombre_del_modelo = "google/flan-t5-large"

pipe = pipeline(
    "text2text-generation",
    model=nombre_del_modelo,
    tokenizer=nombre_del_modelo,
    max_new_tokens=150,
    temperature=0.8
)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Cargamos las librerías de LangChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFacePipeline

# Envolvemos el modelo de Hugging Face dentro de un objeto LangChain,
# para que pueda usarse en la cadena de pregunta-respuesta de QA Chain
modelo_llm = HuggingFacePipeline(pipeline=pipe)

#Creamos la cadena de pregunta-respuesta de QA Chain (LangChain)
chain = load_qa_chain(modelo_llm, chain_type="stuff") # stuff--> concatena todos los textos y los mete en el prompt junto con la pregunta, map_reduce--> resume los textos

/tmp/ipython-input-2126262137.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  modelo_llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-2126262137.py:10: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on

In [ ]:
# Pregunta 1
pregunta = "Què és la distància euclidiana?"
"""
LangChain genera el embedding de la pregunta y busca en FAISS los documentos más cercanos semánticamente
"""
# Obtenemos las 3 porciones del documento más relevantes con la pregunta
docs = knowledge_base.similarity_search(pregunta, 3)
# Con la cadena enlazamos el LLM con la pregunta y las porciones más relevantes del texto
resposta = chain.invoke({"input_documents": docs,"question": pregunta})# invoke--> envía el prompt+docs al LLM y recibe la respuesta del modelo
print(f"Resposta: {resposta['output_text']}")


Resposta: una mètrica til en nombroses aplicacions


In [ ]:
# Pregunta 2
pregunta = "Què és la correlació de Pearson?"
# Obtenemos las 3 porciones del documento más relevantes con la pregunta
docs = knowledge_base.similarity_search(pregunta, 3)
# Con la cadena enlacamos el LLM con la pregunta y las porciones más relevantes del texto
resposta = chain.invoke({"input_documents": docs,"question": pregunta})
print(f"Resposta: {resposta['output_text']}")

Resposta: mesura de similitud entre dues variables que resol els problemes de la similitud euclidiana


In [ ]:
# Pregunta 3
pregunta = "Per a què serveix un dendrograma?"
# Obtenemos las 3 porciones del documento más relevantes con la pregunta
docs = knowledge_base.similarity_search(pregunta, 3)
# Con la cadena enlacamos el LLM con la pregunta y las porciones más relevantes del texto
resposta = chain.invoke({"input_documents": docs,"question": pregunta})
print(f"Resposta: {resposta['output_text']}")

Resposta: Per a observar com de fiables són cadascuna de les reconstruccions
